In [1]:
import qubovert as qv
import numpy as np

# See: https://github.com/jtiosue/qubovert/blob/master/notebook_examples/qubovert_tutorial.ipynb

In [2]:
xs = [qv.spin_var('x%d' % i) for i in range(5)]

In [3]:
## Cost function

C = 0.5*xs[0]*xs[1] + 0.2*xs[2] + -0.9*xs[0]*xs[1]*xs[2]*xs[3] + xs[4]

C

{('x0', 'x1'): 0.5, ('x2',): 0.2, ('x0', 'x1', 'x2', 'x3'): -0.9, ('x4',): 1}

In [4]:
PUSO = C.to_puso()

brute = PUSO.solve_bruteforce()
brute

{0: 1, 1: -1, 2: -1, 3: 1, 4: -1}

In [5]:
0.5*brute[0]*brute[1] + 0.2*brute[2] + -0.9*brute[0]*brute[1]*brute[2]*brute[3] + brute[4]

-2.6

In [6]:
print(C.num_binary_variables)
print(PUSO.num_binary_variables)

5
5


In [7]:
puso_res = qv.sim.anneal_puso(PUSO, num_anneals=3)
puso_res

[AnnealResult(state={0: -1, 1: 1, 2: -1, 3: 1, 4: -1}, value=-2.6, spin=True),
 AnnealResult(state={0: -1, 1: 1, 2: -1, 3: 1, 4: -1}, value=-2.6, spin=True),
 AnnealResult(state={0: -1, 1: 1, 2: -1, 3: 1, 4: -1}, value=-2.6, spin=True)]

In [8]:
puso_res.best

AnnealResult(state={0: -1, 1: 1, 2: -1, 3: 1, 4: -1}, value=-2.6, spin=True)

In [9]:
Q = C.to_qubo()
print("num PUBO variables", C.num_binary_variables)
print("num QUBO variables", Q.num_binary_variables)
print()
# print(Q)

num PUBO variables 5
num QUBO variables 7



In [10]:
solutions = Q.solve_bruteforce(all_solutions=True)

solutions = [C.convert_solution(x) for x in solutions]
solutions

[{'x0': 1, 'x1': -1, 'x2': -1, 'x3': 1, 'x4': -1},
 {'x0': -1, 'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}]

In [11]:
for s in solutions:
    print("\t", s, "is", "valid" if C.is_solution_valid(s) else "invalid")

	 {'x0': 1, 'x1': -1, 'x2': -1, 'x3': 1, 'x4': -1} is valid
	 {'x0': -1, 'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1} is valid


## trying with noncontextual H!!!!

In [12]:
import numpy as np
from symmer.symplectic import PauliwordOp, QuantumState
from symmer.utils import exact_gs_energy

def place_sites(indices, pauli, N):
    I = ['I']*N
    for i in indices:
        I[i] = pauli
    return ''.join(I)
    
def HeisenbergHam(N, J_X=1, J_Y=1, J_Z=1, h=1):
    constants = {'X':J_X, 'Y':J_Y, 'Z':J_Z, 'h':h}
    H_dict = {}
    for i in range(N-1):
        for P in ['X','Y','Z']:
            H_dict[place_sites([i, i+1], P, N)] = constants[P]
    for i in range(N):
        H_dict[place_sites([i], 'Z', N)] = constants['h']

    return PauliwordOp.from_dictionary(H_dict).cleanup().multiply_by_constant(-1/2).sort()

J_X, J_Y, J_Z, h = np.random.random(4)
H = HeisenbergHam(4, J_X, J_Y, J_Z, h=0)
gs_nrg, gs_psi = exact_gs_energy(H.to_sparse_matrix)
print('The heisenberg Hamiltonian is\n')
print(H); print()
print(f'with ground state energy {gs_nrg} and corresponding eigenvector\n')
print(gs_psi)

The heisenberg Hamiltonian is

-0.273+0.000j IIYY +
-0.273+0.000j IYYI +
-0.273+0.000j YYII +
-0.089+0.000j IIXX +
-0.089+0.000j IXXI +
-0.089+0.000j XXII +
-0.064+0.000j IIZZ +
-0.064+0.000j IZZI +
-0.064+0.000j ZZII

with ground state energy -0.8252540522879801 and corresponding eigenvector

-0.304+0.000j |0000> +
 0.000+0.000j |0001> +
 0.000+0.000j |0010> +
 0.332+0.000j |0011> +
 0.000+0.000j |0100> +
 0.390+0.000j |0101> +
 0.381+0.000j |0110> +
-0.000+0.000j |0111> +
 0.000+0.000j |1000> +
 0.381+0.000j |1001> +
 0.390+0.000j |1010> +
-0.000+0.000j |1011> +
 0.332+0.000j |1100> +
-0.000+0.000j |1101> +
-0.000+0.000j |1110> +
-0.304+0.000j |1111>


In [13]:
from symmer.projection import ContextualSubspace

cs_vqe = ContextualSubspace(H, noncontextual_strategy='SingleSweep_magnitude')

In [14]:
cs_vqe.noncontextual_operator

-0.064+0.000j IIZZ +
-0.064+0.000j ZZII +
-0.089+0.000j IIXX +
-0.273+0.000j IIYY +
-0.273+0.000j IYYI +
-0.089+0.000j XXII +
-0.273+0.000j YYII

In [15]:
from symmer.symplectic import NoncontextualOp

In [16]:
H_noncon = NoncontextualOp.from_PauliwordOp(cs_vqe.noncontextual_operator)


H_noncon

-0.064+0.000j IIZZ +
-0.064+0.000j ZZII +
-0.089+0.000j IIXX +
-0.273+0.000j IIYY +
-0.273+0.000j IYYI +
-0.089+0.000j XXII +
-0.273+0.000j YYII

In [17]:
H_noncon.symmetry_generators

 1 ZZZZ 
 1 IIYY 
 1 XXZZ

In [18]:
H_noncon.clique_operator

 1.000+0.000j IYYI +
 1.000+0.000j IIZZ

In [19]:
q_vec = np.ones(H_noncon.symmetry_generators.n_terms)
q_vec

array([1., 1., 1.])

In [20]:
r_vec = np.array(range(1, H_noncon.symmetry_generators.n_terms))
r_vec = r_vec/np.linalg.norm(r_vec)

# r_vec = r_vec.reshape([-1,1])
r_vec

array([0.4472136 , 0.89442719])

In [21]:
H_noncon.noncontextual_objective_function(q_vec, r_vec)

-0.2358018065657227

In [22]:
H_noncon.solve(strategy='brute_force')
H_noncon.energy

-0.8243786561190207

In [23]:
r_vec = H_noncon.clique_operator.coeff_vec

In [24]:
r_part = np.sum(H_noncon.r_indices * r_vec, axis=1)
r_part[np.where(r_part==0)]=1 # set all zero terms to 1 (aka multiply be value of 1)
q_vec_SPIN = [qv.spin_var('x%d' % i) for i in range(H_noncon.symmetry_generators.n_terms)]

COST=0
for P_index, term in enumerate(H_noncon.G_indices):
    non_zero_inds = term.nonzero()[0]
    
    # collect all the spin terms
    G_term=1
    for i in non_zero_inds:
        G_term *= q_vec_SPIN[i]
    
    # cost function
    COST+=G_term * H_noncon.coeff_vec[P_index].real * H_noncon.pauli_mult_signs[P_index] * r_part[P_index].real

In [25]:
spin_problem = COST.to_puso()

brute = spin_problem.solve_bruteforce()
brute

{0: 1, 1: 1, 2: -1}

In [26]:
solution = COST.convert_solution(brute)
COST.value(solution)

-0.8243786561190207

In [27]:
spin_problem = COST.to_puso()

brute = spin_problem.solve_bruteforce()
brute

{0: 1, 1: 1, 2: -1}

In [28]:
puso_res = qv.sim.anneal_quso(spin_problem, num_anneals=3)
anneal = puso_res.best.state

In [29]:
solution_anneal = COST.convert_solution(anneal)
COST.value(solution_anneal)

-0.8243786561190207

In [30]:
# Symmer version!

In [31]:
# brute_force_PUSO
energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='P')
energy - H_noncon.energy

2.973177259946169e-13

In [32]:
# brute_force_PUSO
energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='P')
energy - H_noncon.energy

9.992007221626409e-16

In [33]:
# brute_force_QUSO
energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='Q')
energy- H_noncon.energy

-1.1102230246251565e-16

In [34]:
# annealing_QUSO
energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='Q', num_anneals=100)
energy-H_noncon.energy

0.5230692213380259

In [35]:
# annealing_PUSO
energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='P', num_anneals=100)
energy-H_noncon.energy

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=num_anneals)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symmer/symplectic/noncontextual_op.py:440: QUBOVertWarning: The input problem has degree <= 2; consider using

2.3314683517128287e-15